In [150]:
import quimb.tensor as qtn
import quimb as qu
import numpy as np
import math
import random

In [151]:
N = 64
s = ""
for i in range(0, N):
    r = random.randint(0, 1)
    if r == 0:
        s = s + "0"
    else:
        s = s + "1"
print(s)
compress = False
def check_bit_in_s(s, i, k):
    return (s[k] == i)

1101010001111011100100100111010110001001110000010001110011111000


In [152]:
hadamard_matrix = np.array([[1,1],[1,-1]]) * 1/np.sqrt(2)
hadamard_tensors = []
for m in range(0, N+1):
    hadamard_tensors.append(qtn.Tensor(data=hadamard_matrix, inds=["hi_{}".format(m), "hj_{}".format(m)], tags=("I_{}".format(m))))

hadamard_tn = qtn.TensorNetwork(hadamard_tensors)

hadamard_mpo = qtn.MatrixProductOperator.from_TN(hadamard_tn,
site_tag_id='I_{}',
    upper_ind_id='hj_{}',
    lower_ind_id='hi_{}',
    cyclic=False,
    L=N+1)

In [153]:
# Oracle

to0 = np.zeros((2,2,2))
to0[0][0][0] = 1
to0[1][1][0] = (1 - check_bit_in_s(s, '1', 0))
to0[1][1][1] = check_bit_in_s(s, '1', 0)

to_datas = [to0]

for i in range(1, N):
    t_k = np.zeros((2,2,2,2))
    t_k[0][0][0][0] = 1
    t_k[0][1][1][0] = (1 - check_bit_in_s(s, '1', i))
    t_k[0][1][1][1] = check_bit_in_s(s, '1', i)
    t_k[1][0][0][1] = 1
    t_k[1][1][1][0] = check_bit_in_s(s, '1', i)
    t_k[1][1][1][1] = (1 - check_bit_in_s(s, '1', i))
    to_datas.append(t_k)

ton = np.zeros((2,2,2))
ton[0][0][0] = 1
ton[0][1][1] = 1
ton[1][0][1] = 1
ton[1][1][0] = 1

to_datas.append(ton)

oracle_tensors = []

oracle_tensor_0 = qtn.Tensor(data=to0, inds=["oi_0", "oj_0","0"], tags=("I_0"))
oracle_tensors.append(oracle_tensor_0)

for i in range(1, N):
    oracle_tensors.append(qtn.Tensor(data=to_datas[i], inds=["{}".format(i-1), "oi_{}".format(i), "oj_{}".format(i), "{}".format(i)], tags=("I_{}".format(i))))

oracle_tensor_n = qtn.Tensor(data=ton, inds=["{}".format(N-1), "oi_{}".format(N), "oj_{}".format(N)], tags=("I_{}".format(N)))
oracle_tensors.append(oracle_tensor_n)
oracle_tn = qtn.TensorNetwork(oracle_tensors)
# oracle_tn.draw(show_inds='all', show_tags = True)

In [154]:
oracle_mpo = qtn.MatrixProductOperator.from_TN(oracle_tn,
site_tag_id='I_{}',
    upper_ind_id='oj_{}',
    lower_ind_id='oi_{}',
    cyclic=False,
    L=N+1)

In [155]:
# Init State

init_t0 = np.array([1, 0])

init_tensors = []

for i in range(0, N+1):
    init_tensors.append(qtn.Tensor(data=init_t0, inds=["ii_{}".format(i)], tags=("I_{}".format(i))))

init_tn = qtn.TensorNetwork(init_tensors)
# init_tn.draw(show_inds='all', show_tags = True)

init_mps = qtn.MatrixProductState.from_TN(init_tn,
site_ind_id='ii_{}',
site_tag_id='I_{}',
    cyclic=False,
    L=N+1)

In [156]:
x_matrix = np.zeros((2, 2))
x_matrix[0][1] = 1
x_matrix[1][0] = 1
id_matrix = np.zeros((2, 2))
id_matrix[0][0] = 1
id_matrix[1][1] = 1
x_tensors = [qtn.Tensor(data=id_matrix, inds=["xi_{}".format(i), "xj_{}".format(i)], tags=("I_{}".format(i))) for i in range(0, N)]
x_tensors.append(qtn.Tensor(data=x_matrix, inds=["xi_{}".format(N), "xj_{}".format(N)], tags=("I_{}".format(N))))
x_tn = qtn.TensorNetwork(x_tensors)
x_mpo = qtn.MatrixProductOperator.from_TN(x_tn,
site_tag_id='I_{}',
    upper_ind_id='xj_{}',
    lower_ind_id='xi_{}',
    cyclic=False,
    L=N+1)

In [157]:
out = x_mpo.apply(init_mps, compress=compress)
out = hadamard_mpo.apply(out, compress=compress)
out = oracle_mpo.apply(out, compress=compress)
out = hadamard_mpo.apply(out, compress=compress)


In [158]:
circ = qtn.Circuit(psi0=out)

In [159]:
sampled_s = ""
is_correct = False
print(s)
for b in circ.sample(100):
    print(b[:-1])
    if b[:-1] == s:
        sampled_s = b[:-1]
        is_correct = True
        break
print(is_correct, s, sampled_s)

1101010001111011100100100111010110001001110000010001110011111000
1101010001111011100100100111010110001001110000010001110011111000
True 1101010001111011100100100111010110001001110000010001110011111000 1101010001111011100100100111010110001001110000010001110011111000


In [160]:


# is_correct = False
# sampled_s = ""
# for i in range(0,10):
#     index = np.random.choice(len(result), p=result)
#     index_s = bin(index)[2:].zfill(N)
#     # print(index_s)
#     if index_s == s:
#         is_correct = True 
#         sampled_s = index_s
#         break

# print(is_correct, s, sampled_s)